In [ ]:
'''
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
    return list(rows)
 
    #for row in rows:
     #   print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        return select_all_tasks(conn)
''' 

In [ ]:
'''main()
data=main()
type(data)
data

In [1]:
#I thought this way of connecting was easier to work with
!pip install ipython-sql
%load_ext sql
%sql sqlite:///sqlite_db_pythonsqlite.db

Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

In [2]:
%sql SELECT * FROM FACILITIES WHERE membercost IS NOT 0

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
4,Massage Room 1,9.9,80,4000,3000
5,Massage Room 2,9.9,80,4000,3000
6,Squash Court,3.5,17.5,5000,80


Q2: How many facilities do not charge a fee to members?

In [3]:
%sql SELECT COUNT(*) FROM FACILITIES WHERE membercost IS NOT 0

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


COUNT(*)
5


Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [4]:
%sql SELECT facid, name, membercost, monthlymaintenance FROM FACILITIES WHERE membercost IS NOT 0 AND membercost < monthlymaintenance*.2

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5,200
1,Tennis Court 2,5,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [5]:
%sql SELECT * FROM FACILITIES WHERE facid IN (1,5)

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5,25,8000,200
5,Massage Room 2,9.9,80,4000,3000


Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [6]:
%%sql SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance <100 THEN "cheap"
    WHEN monthlymaintenance >100 THEN "expensive"
    END AS cost
FROM FACILITIES

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


name,monthlymaintenance,cost
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [7]:
%sql SELECT firstname, surname FROM MEMBERS WHERE joindate=(SELECT max(joindate) FROM MEMBERS)

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


firstname,surname
Darren,Smith


Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [8]:
%%sql SELECT firstname || ' ' || surname AS member, f.name
FROM MEMBERS as m
INNER JOIN BOOKINGS as b
ON m.memid=b.memid
INNER JOIN FACILITIES as f
ON f.facid=b.facid WHERE f.name LIKE "Tennis Court%"
GROUP BY member
ORDER BY member

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


member,name
Anne Baker,Tennis Court 1
Burton Tracy,Tennis Court 2
Charles Owen,Tennis Court 1
Darren Smith,Tennis Court 2
David Farrell,Tennis Court 1
David Jones,Tennis Court 2
David Pinker,Tennis Court 1
Douglas Jones,Tennis Court 1
Erica Crumpet,Tennis Court 1
Florence Bader,Tennis Court 2


Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [9]:
%%sql SELECT f.name, m.firstname || ' ' || m.surname AS customer,
CASE WHEN b.memid>0 THEN b.slots*f.membercost
    WHEN b.memid=0 THEN b.slots*f.guestcost
    END AS cost
FROM MEMBERS as m
INNER JOIN BOOKINGS as b
ON m.memid=b.memid
INNER JOIN FACILITIES as f
ON f.facid=b.facid 
WHERE b.starttime LIKE "2012-09-14%" AND cost >30
ORDER BY cost DESC

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


name,customer,cost
Massage Room 2,GUEST GUEST,320
Massage Room 1,GUEST GUEST,160
Massage Room 1,GUEST GUEST,160
Massage Room 1,GUEST GUEST,160
Tennis Court 2,GUEST GUEST,150
Tennis Court 1,GUEST GUEST,75
Tennis Court 1,GUEST GUEST,75
Tennis Court 2,GUEST GUEST,75
Squash Court,GUEST GUEST,70.0
Massage Room 1,Jemima Farrell,39.6


Q9: This time, produce the same result as in Q8, but using a subquery.

In [10]:
%%sql SELECT f.name, m.firstname || ' ' || m.surname AS customer,
CASE 
    WHEN 
        (SELECT b.memid 
         FROM BOOKINGS
        ) >0 THEN b.slots*f.membercost
    ELSE
        b.slots*f.guestcost
    END AS cost
FROM MEMBERS as m
INNER JOIN BOOKINGS as b
ON m.memid=b.memid
INNER JOIN FACILITIES as f
ON f.facid=b.facid 
WHERE b.starttime LIKE "2012-09-14%" AND cost >30
ORDER BY cost DESC

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


name,customer,cost
Massage Room 2,GUEST GUEST,320
Massage Room 1,GUEST GUEST,160
Massage Room 1,GUEST GUEST,160
Massage Room 1,GUEST GUEST,160
Tennis Court 2,GUEST GUEST,150
Tennis Court 1,GUEST GUEST,75
Tennis Court 1,GUEST GUEST,75
Tennis Court 2,GUEST GUEST,75
Squash Court,GUEST GUEST,70.0
Massage Room 1,Jemima Farrell,39.6


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [11]:
%%sql SELECT f.name, 
    SUM(CASE 
        WHEN b.memid>0 THEN b.slots*f.membercost
        WHEN b.memid=0 THEN b.slots*f.guestcost
    END) AS totalrevenue
FROM BOOKINGS AS b
INNER JOIN FACILITIES AS f
ON b.facid=f.facid 
GROUP BY f.name
HAVING totalrevenue <1000
ORDER BY totalrevenue

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


name,totalrevenue
Table Tennis,180
Snooker Table,240
Pool Table,270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [12]:
%%sql 
SELECT customer.surname || ', ' || customer.firstname AS customer, recommend.surname || ', ' || recommend.firstname AS referralsource
FROM MEMBERS customer, MEMBERS recommend
WHERE customer.recommendedby = recommend.memid
ORDER BY customer

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


customer,referralsource
"Bader, Florence","Stibbons, Ponder"
"Baker, Anne","Stibbons, Ponder"
"Baker, Timothy","Farrell, Jemima"
"Boothe, Tim","Rownam, Tim"
"Butters, Gerald","Smith, Darren"
"Coplin, Joan","Baker, Timothy"
"Crumpet, Erica","Smith, Tracy"
"Dare, Nancy","Joplette, Janice"
"Genting, Matthew","Butters, Gerald"
"Hunt, John","Purview, Millicent"


Q12: Find the facilities with their usage by member, but not guests

In [13]:
%%sql SELECT f.name, SUM(b.slots) AS slotsbooked
FROM FACILITIES AS f
INNER JOIN BOOKINGS AS b
ON f.facid=b.facid
WHERE b.memid <> 0
GROUP BY f.name
ORDER BY f.name

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


name,slotsbooked
Badminton Court,1086
Massage Room 1,884
Massage Room 2,54
Pool Table,856
Snooker Table,860
Squash Court,418
Table Tennis,794
Tennis Court 1,957
Tennis Court 2,882


Q13: Find the facilities usage by month, but not guests

In [14]:
%%sql 
SELECT f.name, SUM(b.slots) AS slotsbooked, strftime('%Y-%m',b.starttime) AS month
FROM FACILITIES AS f
INNER JOIN BOOKINGS AS b
ON f.facid=b.facid
WHERE b.memid <> 0
GROUP BY f.name
ORDER BY f.name

 * sqlite:///sqlite_db_pythonsqlite.db
Done.


name,slotsbooked,month
Badminton Court,1086,2012-07
Massage Room 1,884,2012-07
Massage Room 2,54,2012-07
Pool Table,856,2012-07
Snooker Table,860,2012-07
Squash Court,418,2012-07
Table Tennis,794,2012-07
Tennis Court 1,957,2012-07
Tennis Court 2,882,2012-07
